In [ ]:
import os
import numpy as np
import cv2


DATASET_PATH = "dataset"  
IMG_SIZE = (128, 128)  
CLASSES = ["Light", "Very Light", "Medium","Deep", "Very Deep"]  

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:  return None 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, IMG_SIZE)  # Resize to uniform size
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_dataset():
    images = []
    labels = []
    for class_name in CLASSES:
        class_path = os.path.join(DATASET_PATH, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = preprocess_image(img_path)
            images.append(img)
            labels.append(class_name)
    return np.array(images), np.array(labels)


print("Loading and preprocessing dataset...")
images, labels = load_dataset()
print("Data Loaded!!")

Loading and preprocessing dataset...


In [3]:
images.shape

(1489, 128, 128, 3)

In [4]:
INPUT_SIZE = 128 * 128 * 3
HIDDEN_SIZE = 64
OUTPUT_SIZE = len(CLASSES)          
LEARNING_RATE = 0.01
EPOCHS = 25
BATCH_SIZE = 32


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)


def one_hot(labels, class_list):
    mapping = {c: i for i, c in enumerate(class_list)}
    y = np.zeros((len(labels), len(class_list)))
    for idx, lab in enumerate(labels):
        y[idx, mapping[lab]] = 1
    return y

class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size):
        # Xavier/Glorot initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / (input_size + hidden_size))
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / (hidden_size + output_size))
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        # Layer 1
        self.Z1 = X.dot(self.W1) + self.b1          # pre-activation
        self.A1 = sigmoid(self.Z1)                  # activation
        
        # Layer 2
        self.Z2 = self.A1.dot(self.W2) + self.b2     # pre-activation
        self.A2 = softmax(self.Z2)                   # output probabilities
        return self.A2

    def compute_loss(self, Y_pred, Y_true):
        # Cross-entropy loss
        m = Y_true.shape[0]
        log_likelihood = -np.log(Y_pred + 1e-9) * Y_true
        loss = np.sum(log_likelihood) / m
        return loss

    def backward(self, X, Y_true):
        m = Y_true.shape[0]
        
        # dZ2 = A2 - Y
        dZ2 = (self.A2 - Y_true) / m
        # gradients W2, b2
        dW2 = self.A1.T.dot(dZ2)
        db2 = np.sum(dZ2, axis=0, keepdims=True)
        
        # backprop into hidden layer
        dA1 = dZ2.dot(self.W2.T)
        dZ1 = dA1 * sigmoid_derivative(self.Z1)
        # gradients W1, b1
        dW1 = X.T.dot(dZ1)
        db1 = np.sum(dZ1, axis=0, keepdims=True)
        
        self.W2 -= LEARNING_RATE * dW2
        self.b2 -= LEARNING_RATE * db2
        self.W1 -= LEARNING_RATE * dW1
        self.b1 -= LEARNING_RATE * db1

    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)

def get_batches(X, Y, batch_size):
    for i in range(0, X.shape[0], batch_size):
        yield X[i:i+batch_size], Y[i:i+batch_size]


# images, labels = load_dataset()  
X = images.reshape(len(images), -1)  # flatten
Y = one_hot(labels, CLASSES)

perm = np.random.permutation(X.shape[0])
X, Y = X[perm], Y[perm]


In [6]:
split_index = int(0.7 * len(X))
X_train, Y_train = X[:split_index], Y[:split_index]
X_test, Y_test = X[split_index:], Y[split_index:]


nn = SimpleNN(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

for epoch in range(1, EPOCHS + 1):
    epoch_loss = 0
    for X_batch, Y_batch in get_batches(X_train, Y_train, BATCH_SIZE):
        Y_pred = nn.forward(X_batch)
        loss = nn.compute_loss(Y_pred, Y_batch)
        nn.backward(X_batch, Y_batch)
        epoch_loss += loss * X_batch.shape[0]
    epoch_loss /= X_train.shape[0]
    print(f"Epoch {epoch}/{EPOCHS} — Loss: {epoch_loss:.4f}")


train_preds = nn.predict(X_train)
train_acc = np.mean(train_preds == np.argmax(Y_train, axis=1))
print(f"Training Accuracy on 70% data: {train_acc * 100:.2f}%")

Epoch 1/25 — Loss: 1.4988
Epoch 2/25 — Loss: 1.2463
Epoch 3/25 — Loss: 1.1151
Epoch 4/25 — Loss: 1.0236
Epoch 5/25 — Loss: 0.9539
Epoch 6/25 — Loss: 0.8990
Epoch 7/25 — Loss: 0.8534
Epoch 8/25 — Loss: 0.8141
Epoch 9/25 — Loss: 0.7790
Epoch 10/25 — Loss: 0.7470
Epoch 11/25 — Loss: 0.7174
Epoch 12/25 — Loss: 0.6897
Epoch 13/25 — Loss: 0.6637
Epoch 14/25 — Loss: 0.6392
Epoch 15/25 — Loss: 0.6160
Epoch 16/25 — Loss: 0.5940
Epoch 17/25 — Loss: 0.5731
Epoch 18/25 — Loss: 0.5533
Epoch 19/25 — Loss: 0.5345
Epoch 20/25 — Loss: 0.5165
Epoch 21/25 — Loss: 0.4993
Epoch 22/25 — Loss: 0.4829
Epoch 23/25 — Loss: 0.4672
Epoch 24/25 — Loss: 0.4523
Epoch 25/25 — Loss: 0.4380
Training Accuracy on 70% data: 89.92%


In [32]:
# Predict on test set
test_preds = nn.predict(X_test)
test_labels = np.argmax(Y_test, axis=1)
test_acc = np.mean(test_preds == test_labels)
print(f"Test Accuracy on 30% data: {test_acc * 100:.2f}%")


Test Accuracy on 30% data: 80.09%


In [1]:
UNKNOWN_PATH = "unknown_images"  # Folder with images of unknown skin tones
output_log_path = "predictions_log.txt"

unknown_images = []
img_names = []

for file in os.listdir(UNKNOWN_PATH):
    img_path = os.path.join(UNKNOWN_PATH, file)
    img = preprocess_image(img_path)
    if img is not None:
        unknown_images.append(img)
        img_names.append(file)

unknown_images = np.array(unknown_images).reshape(len(unknown_images), -1)

\

probs = nn.forward(unknown_images)
predictions = np.argmax(probs, axis=1)
confidences = np.max(probs, axis=1)


with open(output_log_path, "w") as f:
    for name, pred, conf in zip(img_names, predictions, confidences):
        f.write(f"{name}: Predicted Skin Tone = {CLASSES[pred]} | Confidence = {conf:.4f}\n")

print(f"Prediction log saved to: {output_log_path}")


NameError: name 'os' is not defined